In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import matplotlib.ticker as mtick
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
from shapely.geometry import Polygon

import geopandas as gpd
from shapely.geometry import Polygon
from itertools import product
import warnings
warnings.filterwarnings("ignore")

import shapely

In [2]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

In [3]:
df.head()

,agricultureStructureIndicator,asOfDate,basementEnclosureCrawlspaceType,policyCount,crsClassificationCode,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,baseFloodElevation,ratedFloodZone,houseWorship,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,obstructionType,occupancyType,originalConstructionDate,originalNBDate,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss,primaryResidenceIndicator,buildingDamageAmount,buildingDeductibleCode,netBuildingPaymentAmount,buildingPropertyValue,causeOfDamage,condominiumCoverageTypeCode,contentsDamageAmount,contentsDeductibleCode,netContentsPaymentAmount,contentsPropertyValue,disasterAssistanceCoverageRequired,eventDesignationNumber,ficoNumber,floodCharacteristicsIndicator,floodWaterDuration,floodproofedIndicator,floodEvent,iccCoverage,netIccPaymentAmount,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,nonPaymentReasonContents,nonPaymentReasonBuilding,numberOfUnits,buildingReplacementCost,contentsReplacementCost,replacementCostBasis,stateOwnedIndicator,waterDepth,floodZoneCurrent,buildingDescriptionCode,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
0,0,2020-01-22T16:55:53.000Z,NaN,1,8.0,1998-02-07T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,4.0,0,10.0,1.0,1963-01-01T00:00:00.000Z,1997-01-11T00:00:00.000Z,NaN,NaN,NaN,0,7,0,200000.0,50000.0,1998,0,382.0,0,0.00,937.0,1,N,NaN,0,0.0,NaN,0.0,None,612.0,NaN,0.0,0,Pineapple Express - Southern,15000.0,0.0,60294.0,None,"OCEANSIDE, CITY OF",97.0,1.0,1.0,937.0,NaN,A,0,0.0,None,NaN,0,CA,Currently Unavailable,92056.0,6073.0,6.073019e+09,6.073019e+10,33.2,-117.3,4053d562-94c7-4155-9d60-3030f34a7abc
1,0,2020-01-22T16:55:53.000Z,NaN,1,8.0,2005-08-29T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,2.0,0,NaN,1.0,1967-07-01T00:00:00.000Z,1990-07-12T00:00:00.000Z,NaN,NaN,NaN,0,7,0,100000.0,40000.0,2005,1,NaN,0,0.00,NaN,1,N,NaN,0,0.0,NaN,0.0,None,654.0,NaN,0.0,0,Hurricane Katrina,30000.0,0.0,225203.0,None,NEW ORLEANS/ORLEANS PARISH*,6.0,6.0,1.0,NaN,NaN,A,0,0.0,None,NaN,0,LA,Currently Unavailable,70131.0,22071.0,2.207100e+10,2.207100e+11,29.9,-90.0,4094445e-bc54-42b2-a60d-b949bdc2df3a
2,0,2020-01-22T16:55:53.000Z,NaN,1,9.0,1998-09-28T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,1.0,0,10.0,1.0,1972-01-01T00:00:00.000Z,1997-07-24T00:00:00.000Z,8813.21,1720.0,0.0,0,1,0,100000.0,50000.0,1998,1,9313.0,0,8813.21,80000.0,1,N,2220.0,0,1720.0,0.0,0.0,None,133.0,NaN,0.0,0,Hurricane Georges (Panhandle),15000.0,0.0,120274.0,None,SANTA ROSA COUNTY *,NaN,NaN,1.0,100000.0,0.0,A,0,0.0,None,NaN,0,FL,Currently Unavailable,32566.0,12113.0,1.211301e+10,1.211301e+11,30.4,-86.9,659ba8e0-1bb3-4442-ad57-3f384db43176
3,0,2019-09-19T13:45:58.000Z,1.0,1,9.0,1994-10-07T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,2.0,0,10.0,1.0,1960-01-01T00:00:00.000Z,1993-10-01T00:00:00.000Z,2906.00,0.0,0.0,0,7,0,100000.0,25000.0,1994,0,4428.0,0,2906.00,100000.0,1,N,NaN,0,0.0,NaN,0.0,None,NaN,NaN,0.0,0,None,NaN,0.0,450026.0,None,"BEAUFORT, CITY OF",97.0,NaN,1.0,0.0,NaN,A,0,0.0,None,NaN,0,SC,Currently Unavailable,29902.0,45013.0,4.501300e+10,4.501300e+11,32.4,-80.7,97063578-59b6-403d-b9c3-2e70fd8b6f38
4,0,2019-09-19T13:45:58.000Z,NaN,1,8.0,1996-03-11T00:00:00.000Z,0,None,NaN,NaN,X,0,NaN,NaN,NaN,1.0,0,NaN,1.0,1988-01-01T00:00:00.000Z,1996-01-11T00:00:00.000Z,3875.53,1545.0,0.0,1,7,0,100000.0,25000.0,1996,1,5252.0,0,3875.53,100000.0,1,N,3115.0,0,1545.0,0.0,0.0,None,NaN,NaN,0.0,0,None,NaN,0.0,125092.0,None,BREVARD COUNTY *,NaN,NaN,1.0,0.0,0.0,A,0,0.0,None,1.0,0,FL,Currently Unavailable,32940.0,12009.0,1.200906e+10,1.200906e+11,28.3,-80.7,613c20ef-7838-48ee-a257-07a49e997d3a


In [4]:
#fill na values with 0

df_geographic_unique = df[['state', 'reportedZipCode', 'countyCode', 'censusTract', 'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss']].drop_duplicates()

df_geographic_unique['reportedZipCode'] = df_geographic_unique['reportedZipCode'].fillna(0)
df_geographic_unique['countyCode'] = df_geographic_unique['countyCode'].fillna(0)
df_geographic_unique['censusTract'] = df_geographic_unique['censusTract'].fillna(0)
df_geographic_unique['censusBlockGroupFips'] = df_geographic_unique['censusBlockGroupFips'].fillna(0)
df_geographic_unique['latitude'] = df_geographic_unique['latitude'].fillna(0)
df_geographic_unique['longitude'] = df_geographic_unique['longitude'].fillna(0)

df_geographic_unique = df_geographic_unique.drop_duplicates()

In [5]:
#convert the units to string to use mapping and fix the lenght of each units

df_geographic_unique['reportedZipCode'] = df_geographic_unique['reportedZipCode'].astype(int).astype(str)
df_geographic_unique['reportedZipCode'] = [zipcode.zfill(5) for zipcode in df_geographic_unique['reportedZipCode']]

df_geographic_unique['censusBlockGroupFips'] = [str(int(float(i))) for i in df_geographic_unique['censusBlockGroupFips']]
df_geographic_unique['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_geographic_unique['censusBlockGroupFips']]

df_geographic_unique['countyCode'] = [str(int(float(i))) for i in df_geographic_unique['countyCode']]
df_geographic_unique['countyCode'] = [censusBG.zfill(5) for censusBG in df_geographic_unique['countyCode']]

df_geographic_unique['censusTract'] = [str(int(float(i))) for i in df_geographic_unique['censusTract']]
df_geographic_unique['censusTract'] = [censusBG.zfill(11) for censusBG in df_geographic_unique['censusTract']]

In [6]:
# Define bins and labels for yearOfLoss_1990_2021

bins_1990_2021 = [df_geographic_unique['yearOfLoss'].min(), 2000, 2010, 2020, df_geographic_unique['yearOfLoss'].max() + 1]
labels_1990_2021 = [1990, 2000, 2010, 2020]

df_geographic_unique['yearOfLoss_1990_2021'] = pd.cut(df_geographic_unique['yearOfLoss'], bins=bins_1990_2021, labels=labels_1990_2021, right=False).astype(int)

df_geographic_unique.drop(columns='yearOfLoss', inplace=True)

df_geographic_unique = df_geographic_unique.drop_duplicates()

## Read shapefiles

In [7]:
#State shapefile

states = pygris.states()

state_df = states[['STUSPS', 'NAME', 'geometry']]

Using the default year of 2021


In [8]:
#Lat-Long shapefile
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/lat_long_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
lat_long_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

In [9]:
# Blockgroup shapefile

chunk_size = 40000 
chunks = [x for x in range(0, 120000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/BG_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
BG_df= pd.concat(gdf_list, ignore_index=True)

In [ ]:
# Read shapefile of zipcode

chunk_size = 40000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 80000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/zipcode_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)
    
# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
zipcode_df = pd.concat(gdf_list, ignore_index=True)

In [ ]:
# Read shapefile of county code
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/County_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
County_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

In [ ]:
# Read shapefile of census tract code
chunk_size = 30000 
chunks = [x for x in range(0, 60000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/Tract_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
Tract_df= pd.concat(gdf_list, ignore_index=True)

### Mapping Geometries to the Dataset

In [ ]:
state_df.rename(columns={'geometry': 'geometry_state'}, inplace=True)
lat_long_df.rename(columns={'geometry': 'geometry_lat_long'}, inplace=True)
BG_df.rename(columns={'geometry': 'geometry_BG'}, inplace=True)
zipcode_df.rename(columns={'geometry': 'geometry_zipcode'}, inplace=True)
County_df.rename(columns={'geometry': 'geometry_county'}, inplace=True)
Tract_df.rename(columns={'geometry': 'geometry_tract'}, inplace=True)

In [ ]:
# Setting the multi-index on lat_long_df
lat_long_df.set_index(['latitude', 'longitude'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_lat_long'] = df_geographic_unique.set_index(['latitude', 'longitude']).index.map(lat_long_df['geometry_lat_long'])

lat_long_df.reset_index(inplace=True)

In [ ]:
# Initial mapping with multi-index
BG_df.set_index(['GEOID'], inplace=True)
df_geographic_unique['geometry_BG'] = df_geographic_unique.set_index(['censusBlockGroupFips']).index.map(BG_df['geometry_BG'])

BG_df.reset_index(inplace=True)

In [ ]:
# Initial mapping with multi-index
zipcode_df.set_index(['ZIPcode', 'year'], inplace=True)
df_geographic_unique['geometry_zipcode'] = df_geographic_unique.set_index(['reportedZipCode', 'yearOfLoss_1990_2021']).index.map(zipcode_df['geometry_zipcode'])

zipcode_df.reset_index(inplace=True)

In [ ]:
# Initial mapping with multi-index
state_df.set_index(['STUSPS'], inplace=True)

# Mapping the values
df_geographic_unique['geometry_state'] = df_geographic_unique.set_index(['state']).index.map(state_df['geometry_state'])

state_df.reset_index(inplace=True)

In [ ]:
# Initial mapping with multi-index
County_df.set_index(['CountyID'], inplace=True)
df_geographic_unique['geometry_county'] = df_geographic_unique.set_index(['countyCode']).index.map(County_df['geometry_county'])

# Resetting the index of County_df (return to multi-index)
County_df.reset_index(inplace=True)

In [ ]:
# Initial mapping with multi-index
Tract_df.set_index(['censusTractID'], inplace=True)
df_geographic_unique['geometry_tract'] = df_geographic_unique.set_index(['censusTract']).index.map(Tract_df['geometry_tract'])

# Resetting the index of County_df (return to multi-index)
Tract_df.reset_index(inplace=True)

### Adding the inconsistent flags to the dataset

In [ ]:
df_1_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_1.parquet.gzip')
df_2_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_2.parquet.gzip')
df_3_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_3.parquet.gzip')
df_4_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_4.parquet.gzip')
df_5_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_5.parquet.gzip')
df_6_inconsistent = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/InconsistencyDataframes/inconsistency_dataframe_6.parquet.gzip')

In [ ]:
(sum((df_1_inconsistent['zipInconsistent'] == 1.0) & (df_1_inconsistent['oneWrong'] == 1.0)) + 
 sum((df_3_inconsistent['zipInconsistent'] == 1.0) & (df_3_inconsistent['oneWrong'] == 1.0)) +
 sum((df_5_inconsistent['zipInconsistent'] == 1.0) & (df_5_inconsistent['oneWrong'] == 1.0)))

In [ ]:
(sum((df_1_inconsistent['stateInconsistent'] == 1.0) & (df_1_inconsistent['oneWrong'] == 1.0)) + 
  sum((df_2_inconsistent['stateInconsistent'] == 1.0) & (df_2_inconsistent['oneWrong'] == 1.0)) +
 sum((df_3_inconsistent['stateInconsistent'] == 1.0) & (df_3_inconsistent['oneWrong'] == 1.0)) +
  sum((df_4_inconsistent['stateInconsistent'] == 1.0) & (df_4_inconsistent['oneWrong'] == 1.0)) +
 sum((df_5_inconsistent['stateInconsistent'] == 1.0) & (df_5_inconsistent['oneWrong'] == 1.0)))

In [ ]:
#df_2_inconsistent
df_2_inconsistent['reportedZipCode'] = df_2_inconsistent['reportedZipCode'].fillna(0)

#df_3_inconsistent
df_3_inconsistent['countyCode'] = df_3_inconsistent['countyCode'].fillna(0)

#df_4_inconsistent
df_4_inconsistent['reportedZipCode'] = df_4_inconsistent['reportedZipCode'].fillna(0)
df_4_inconsistent['countyCode'] = df_4_inconsistent['countyCode'].fillna(0)

#df_5_inconsistent
df_5_inconsistent['censusTract'] = df_5_inconsistent['censusTract'].fillna(0)
df_5_inconsistent['censusBlockGroupFips'] = df_5_inconsistent['censusBlockGroupFips'].fillna(0)

In [ ]:
#convert the units to string to use mapping and fix the lenght of each units
#df_2_inconsistent
df_2_inconsistent['reportedZipCode'] = df_2_inconsistent['reportedZipCode'].astype(int).astype(str)
df_2_inconsistent['reportedZipCode'] = [zipcode.zfill(5) for zipcode in df_2_inconsistent['reportedZipCode']]

#df_3_inconsistent
df_3_inconsistent['countyCode'] = [str(int(float(i))) for i in df_3_inconsistent['countyCode']]
df_3_inconsistent['countyCode'] = [censusBG.zfill(5) for censusBG in df_3_inconsistent['countyCode']]

#df_4_inconsistent
df_4_inconsistent['reportedZipCode'] = df_4_inconsistent['reportedZipCode'].astype(int).astype(str)
df_4_inconsistent['reportedZipCode'] = [zipcode.zfill(5) for zipcode in df_4_inconsistent['reportedZipCode']]

df_4_inconsistent['countyCode'] = [str(int(float(i))) for i in df_4_inconsistent['countyCode']]
df_4_inconsistent['countyCode'] = [censusBG.zfill(5) for censusBG in df_4_inconsistent['countyCode']]

#df_5_inconsistent
df_5_inconsistent['censusBlockGroupFips'] = [str(int(float(i))) for i in df_5_inconsistent['censusBlockGroupFips']]
df_5_inconsistent['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_5_inconsistent['censusBlockGroupFips']]

df_5_inconsistent['censusTract'] = [str(int(float(i))) for i in df_5_inconsistent['censusTract']]
df_5_inconsistent['censusTract'] = [censusBG.zfill(11) for censusBG in df_5_inconsistent['censusTract']]

In [ ]:
df_1_inconsistent.drop(columns=['zip_geometry','lat_long_geometry','bg_geometry'], inplace=True)
df_2_inconsistent.drop(columns=['lat_long_geometry','state_geometry','county_geometry'], inplace=True)
df_3_inconsistent.drop(columns=['state_geometry','zip_geometry'], inplace=True)

In [ ]:
# Initial mapping with multi-index
df_1_inconsistent.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'year'], inplace=True)

df_geographic_unique['zipInconsistent'] = df_geographic_unique.drop_duplicates(
                    subset=['reportedZipCode', 'censusTract', 'state', 'countyCode', 
                    'censusBlockGroupFips', 'latitude', 'longitude', 'yearOfLoss_1990_2021']).set_index(
                    ['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips', 
                     'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_1_inconsistent['zipInconsistent'])


df_geographic_unique['zipInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_1_inconsistent['zipInconsistent'])
df_geographic_unique['cbgInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_1_inconsistent['cbgInconsistent'])
df_geographic_unique['tractInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_1_inconsistent['tractInconsistent'])
df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_1_inconsistent['stateInconsistent'])
df_geographic_unique['countyInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_1_inconsistent['countyInconsistent'])
df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_1_inconsistent['latlongInconsistent'])
df_geographic_unique['multiple_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_1_inconsistent['multiple'])
df_geographic_unique['single_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_1_inconsistent['oneWrong'])

# Resetting the index of County_df (return to multi-index)
df_1_inconsistent.reset_index(inplace=True)

In [ ]:
df_2_inconsistent.drop_duplicates(subset=['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude','year'], inplace=True)

In [ ]:
# Initial mapping with multi-index
df_2_inconsistent.set_index(['reportedZipCode','censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'year'], inplace=True)

multi_index = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index
cbgInconsistent = pd.Series(multi_index.map(df_2_inconsistent['cbgInconsistent']), index=multi_index)
tractInconsistent = pd.Series(multi_index.map(df_2_inconsistent['tractInconsistent']), index=multi_index)
stateInconsistent = pd.Series(multi_index.map(df_2_inconsistent['stateInconsistent']), index=multi_index)
countyInconsistent = pd.Series(multi_index.map(df_2_inconsistent['countyInconsistent']), index=multi_index)
latlongInconsistent = pd.Series(multi_index.map(df_2_inconsistent['latlongInconsistent']), index=multi_index)
multiple_inconsistent = pd.Series(multi_index.map(df_2_inconsistent['multiple']), index=multi_index)
single_inconsistent = pd.Series(multi_index.map(df_2_inconsistent['oneWrong']), index=multi_index)

# Revert df_2_inconsistent back to its original form
df_2_inconsistent.reset_index(inplace=True)

# Fill NaN values in df_geographic_unique['cbgInconsistent'] with values from cbgInconsistent Series
df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips', 'latitude', 
                                'longitude', 'yearOfLoss_1990_2021'], inplace=True)
df_geographic_unique['cbgInconsistent'].fillna(cbgInconsistent, inplace=True)
df_geographic_unique['tractInconsistent'].fillna(tractInconsistent, inplace=True)
df_geographic_unique['stateInconsistent'].fillna(stateInconsistent, inplace=True)
df_geographic_unique['countyInconsistent'].fillna(countyInconsistent, inplace=True)
df_geographic_unique['latlongInconsistent'].fillna(latlongInconsistent, inplace=True)
df_geographic_unique['multiple_inconsistent'].fillna(multiple_inconsistent, inplace=True)
df_geographic_unique['single_inconsistent'].fillna(single_inconsistent, inplace=True)

# Reset index for df_geographic_unique if needed
df_geographic_unique.reset_index(inplace=True)

In [ ]:
# # Initial mapping with multi-index
# df_2_inconsistent.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude'], inplace=True)

# df_geographic_unique['cbgInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['cbgInconsistent'])
# df_geographic_unique['tractInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['tractInconsistent'])
# df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['stateInconsistent'])
# df_geographic_unique['countyInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['countyInconsistent'])
# df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['latlongInconsistent'])
# df_geographic_unique['multiple_inconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['multiple'])
# df_geographic_unique['single_inconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['oneWrong'])

# # Resetting the index of County_df (return to multi-index)
# df_2_inconsistent.reset_index(inplace=True)

In [ ]:
# Initial mapping with multi-index
df_3_inconsistent.set_index(['reportedZipCode','censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'year'], inplace=True)

multi_index = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index
zipInconsistent = pd.Series(multi_index.map(df_3_inconsistent['zipInconsistent']), index=multi_index)
cbgInconsistent = pd.Series(multi_index.map(df_3_inconsistent['cbgInconsistent']), index=multi_index)
tractInconsistent = pd.Series(multi_index.map(df_3_inconsistent['tractInconsistent']), index=multi_index)
stateInconsistent = pd.Series(multi_index.map(df_3_inconsistent['stateInconsistent']), index=multi_index)
# countyInconsistent = pd.Series(multi_index.map(df_3_inconsistent['countyInconsistent']), index=multi_index)
latlongInconsistent = pd.Series(multi_index.map(df_3_inconsistent['latlongInconsistent']), index=multi_index)
multiple_inconsistent = pd.Series(multi_index.map(df_3_inconsistent['multiple']), index=multi_index)
single_inconsistent = pd.Series(multi_index.map(df_3_inconsistent['oneWrong']), index=multi_index)

# Revert df_3_inconsistent back to its original form
df_3_inconsistent.reset_index(inplace=True)

# Fill NaN values in df_geographic_unique['cbgInconsistent'] with values from cbgInconsistent Series
df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips', 'latitude', 
                                'longitude', 'yearOfLoss_1990_2021'], inplace=True)
df_geographic_unique['zipInconsistent'].fillna(zipInconsistent, inplace=True)
df_geographic_unique['cbgInconsistent'].fillna(cbgInconsistent, inplace=True)
df_geographic_unique['tractInconsistent'].fillna(tractInconsistent, inplace=True)
df_geographic_unique['stateInconsistent'].fillna(stateInconsistent, inplace=True)
# df_geographic_unique['countyInconsistent'].fillna(countyInconsistent, inplace=True)
df_geographic_unique['latlongInconsistent'].fillna(latlongInconsistent, inplace=True)
df_geographic_unique['multiple_inconsistent'].fillna(multiple_inconsistent, inplace=True)
df_geographic_unique['single_inconsistent'].fillna(single_inconsistent, inplace=True)

# Reset index for df_geographic_unique if needed
df_geographic_unique.reset_index(inplace=True)

In [ ]:
# # Initial mapping with multi-index
# df_3_inconsistent.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'year'], inplace=True)

# df_geographic_unique['zipInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['zipInconsistent'])
# df_geographic_unique['cbgInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['cbgInconsistent'])
# df_geographic_unique['tractInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['tractInconsistent'])
# df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['stateInconsistent'])
# df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['latlongInconsistent'])
# df_geographic_unique['multiple_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['multiple'])
# df_geographic_unique['single_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_3_inconsistent['oneWrong'])

# # Resetting the index of County_df (return to multi-index)
# df_3_inconsistent.reset_index(inplace=True)

In [ ]:
df_4_inconsistent.drop_duplicates(subset=['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude','year'], inplace=True)

In [ ]:
# Initial mapping with multi-index
df_4_inconsistent.set_index(['reportedZipCode','censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'year'], inplace=True)

multi_index = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index
# zipInconsistent = pd.Series(multi_index.map(df_4_inconsistent['zipInconsistent']), index=multi_index)
cbgInconsistent = pd.Series(multi_index.map(df_4_inconsistent['cbgInconsistent']), index=multi_index)
tractInconsistent = pd.Series(multi_index.map(df_4_inconsistent['tractInconsistent']), index=multi_index)
stateInconsistent = pd.Series(multi_index.map(df_4_inconsistent['stateInconsistent']), index=multi_index)
# countyInconsistent = pd.Series(multi_index.map(df_3_inconsistent['countyInconsistent']), index=multi_index)
latlongInconsistent = pd.Series(multi_index.map(df_4_inconsistent['latlongInconsistent']), index=multi_index)
multiple_inconsistent = pd.Series(multi_index.map(df_4_inconsistent['multiple']), index=multi_index)
single_inconsistent = pd.Series(multi_index.map(df_4_inconsistent['oneWrong']), index=multi_index)

# Revert df_4_inconsistent back to its original form
df_4_inconsistent.reset_index(inplace=True)

# Fill NaN values in df_geographic_unique['cbgInconsistent'] with values from cbgInconsistent Series
df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips', 'latitude', 
                                'longitude', 'yearOfLoss_1990_2021'], inplace=True)
# df_geographic_unique['zipInconsistent'].fillna(zipInconsistent, inplace=True)
df_geographic_unique['cbgInconsistent'].fillna(cbgInconsistent, inplace=True)
df_geographic_unique['tractInconsistent'].fillna(tractInconsistent, inplace=True)
df_geographic_unique['stateInconsistent'].fillna(stateInconsistent, inplace=True)
# df_geographic_unique['countyInconsistent'].fillna(countyInconsistent, inplace=True)
df_geographic_unique['latlongInconsistent'].fillna(latlongInconsistent, inplace=True)
df_geographic_unique['multiple_inconsistent'].fillna(multiple_inconsistent, inplace=True)
df_geographic_unique['single_inconsistent'].fillna(single_inconsistent, inplace=True)

# Reset index for df_geographic_unique if needed
df_geographic_unique.reset_index(inplace=True)

In [ ]:
# # Initial mapping with multi-index
# df_4_inconsistent.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude'], inplace=True)

# df_geographic_unique['cbgInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['cbgInconsistent'])
# df_geographic_unique['tractInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['tractInconsistent'])
# df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['stateInconsistent'])
# df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['latlongInconsistent'])
# df_geographic_unique['multiple_inconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['multiple'])
# df_geographic_unique['single_inconsistent'] = df_geographic_unique.set_index(['censusTract', 'state', 'censusBlockGroupFips',
#                             'latitude', 'longitude']).index.map(df_4_inconsistent['oneWrong'])

# # Resetting the index of County_df (return to multi-index)
# df_4_inconsistent.reset_index(inplace=True)

In [ ]:
# Initial mapping with multi-index
df_5_inconsistent.set_index(['reportedZipCode','censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'year'], inplace=True)

multi_index = df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips',
                            'latitude', 'longitude', 'yearOfLoss_1990_2021']).index
zipInconsistent = pd.Series(multi_index.map(df_5_inconsistent['zipInconsistent']), index=multi_index)
# cbgInconsistent = pd.Series(multi_index.map(df_5_inconsistent['cbgInconsistent']), index=multi_index)
# tractInconsistent = pd.Series(multi_index.map(df_5_inconsistent['tractInconsistent']), index=multi_index)
stateInconsistent = pd.Series(multi_index.map(df_5_inconsistent['stateInconsistent']), index=multi_index)
countyInconsistent = pd.Series(multi_index.map(df_5_inconsistent['countyInconsistent']), index=multi_index)
latlongInconsistent = pd.Series(multi_index.map(df_5_inconsistent['latlongInconsistent']), index=multi_index)
multiple_inconsistent = pd.Series(multi_index.map(df_5_inconsistent['multiple']), index=multi_index)
single_inconsistent = pd.Series(multi_index.map(df_5_inconsistent['oneWrong']), index=multi_index)

# Revert df_5_inconsistent back to its original form
df_5_inconsistent.reset_index(inplace=True)

# Fill NaN values in df_geographic_unique['cbgInconsistent'] with values from cbgInconsistent Series
df_geographic_unique.set_index(['reportedZipCode', 'censusTract', 'state', 'countyCode', 'censusBlockGroupFips', 'latitude', 
                                'longitude', 'yearOfLoss_1990_2021'], inplace=True)
df_geographic_unique['zipInconsistent'].fillna(zipInconsistent, inplace=True)
# df_geographic_unique['cbgInconsistent'].fillna(cbgInconsistent, inplace=True)
# df_geographic_unique['tractInconsistent'].fillna(tractInconsistent, inplace=True)
df_geographic_unique['stateInconsistent'].fillna(stateInconsistent, inplace=True)
df_geographic_unique['countyInconsistent'].fillna(countyInconsistent, inplace=True)
df_geographic_unique['latlongInconsistent'].fillna(latlongInconsistent, inplace=True)
df_geographic_unique['multiple_inconsistent'].fillna(multiple_inconsistent, inplace=True)
df_geographic_unique['single_inconsistent'].fillna(single_inconsistent, inplace=True)

# Reset index for df_geographic_unique if needed
df_geographic_unique.reset_index(inplace=True)

In [ ]:
# # Initial mapping with multi-index
# df_5_inconsistent.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'year'], inplace=True)

# df_geographic_unique['zipInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['zipInconsistent'])
# df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['stateInconsistent'])
# df_geographic_unique['countyInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['countyInconsistent'])
# df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['latlongInconsistent'])
# df_geographic_unique['multiple_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['multiple'])
# df_geographic_unique['single_inconsistent'] = df_geographic_unique.set_index(['reportedZipCode', 'state', 'countyCode',
#                             'latitude', 'longitude', 'yearOfLoss_1990_2021']).index.map(df_5_inconsistent['oneWrong'])

# # Resetting the index of County_df (return to multi-index)
# df_5_inconsistent.reset_index(inplace=True)

In [ ]:
# # Initial mapping with multi-index (df_6_inconsistent is empty)
# df_6_inconsistent.set_index(['state', 'countyCode',
#                             'latitude', 'longitude'], inplace=True)

# df_geographic_unique['stateInconsistent'] = df_geographic_unique.set_index(['state', 'countyCode',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['stateInconsistent'])
# df_geographic_unique['countyInconsistent'] = df_geographic_unique.set_index(['state', 'countyCode',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['countyInconsistent'])
# df_geographic_unique['latlongInconsistent'] = df_geographic_unique.set_index(['state', 'countyCode',
#                             'latitude', 'longitude']).index.map(df_2_inconsistent['latlongInconsistent'])

# # Resetting the index of County_df (return to multi-index)
# df_6_inconsistent.reset_index(inplace=True)

In [ ]:
df_geographic_unique[df_geographic_unique['zipInconsistent'].notna()].head()

In [ ]:
test = df_geographic_unique[df_geographic_unique['single_inconsistent'] == 1.0]

print(f"zipInconsistent: {sum(test['zipInconsistent'] == 1.0)}")
print(f"cbgInconsistent: {sum(test['cbgInconsistent'] == 1.0)}")
print(f"tractInconsistent: {sum((test['tractInconsistent'] == 1.0))}")
print(f"stateInconsistent: {sum((test['stateInconsistent'] == 1.0))}")
print(f"countyInconsistent: {sum((test['countyInconsistent'] == 1.0))}")
print(f"latlongInconsistent: {sum((test['latlongInconsistent'] == 1.0))}")
print(f"multiple_inconsistent: {sum((test['multiple_inconsistent'] == 1.0))}")
print(f"single_inconsistent: {sum((test['single_inconsistent'] == 1.0))}")

In [ ]:
print(f"zipInconsistent: {sum((df_geographic_unique['zipInconsistent'].notna()) & (df_geographic_unique['zipInconsistent'] == 1.0))}")
print(f"cbgInconsistent: {sum((df_geographic_unique['cbgInconsistent'].notna()) & (df_geographic_unique['cbgInconsistent'] == 1.0))}")
print(f"tractInconsistent: {sum((df_geographic_unique['tractInconsistent'].notna()) & (df_geographic_unique['tractInconsistent'] == 1.0))}")
print(f"stateInconsistent: {sum((df_geographic_unique['stateInconsistent'].notna()) & (df_geographic_unique['stateInconsistent'] == 1.0))}")
print(f"countyInconsistent: {sum((df_geographic_unique['countyInconsistent'].notna()) & (df_geographic_unique['countyInconsistent'] == 1.0))}")
print(f"latlongInconsistent: {sum((df_geographic_unique['latlongInconsistent'].notna()) & (df_geographic_unique['latlongInconsistent'] == 1.0))}")
print(f"multiple_inconsistent: {sum((df_geographic_unique['multiple_inconsistent'].notna()) & (df_geographic_unique['multiple_inconsistent'] == 1.0))}")
print(f"single_inconsistent: {sum((df_geographic_unique['single_inconsistent'].notna()) & (df_geographic_unique['single_inconsistent'] == 1.0))}")

In [ ]:
#TODO: for all NA values change it to 1
#TODO: create 'multipleInconsistent' column where add all flags
#TODO: for state = 'UN', make it N
#TODO (Not here but later): for missing geometries, change flag to M

In [ ]:
df_geographic_unique['zipInconsistent'].value_counts()

In [ ]:
df_geographic_unique.head()

In [ ]:
df_geographic_unique.to_parquet(

In [ ]:
count_none =(df_geographic_unique['geometry_lat_long'] == None).sum()
print(count_none)

In [ ]:
def is_empty_or_none(geometry):
    return geometry is None or geometry.is_empty


In [67]:
assignment_counts = {
    "bg_zip_not_empty": 0,
    "bg_no_zip_before_2000": 0,
    "no_bg_zip_after_2000": 0,
    "no_bg_no_zip": 0,
    "lat_long_none": 0,
    "lat_long_bg_no_zip_before_2000_1": 0,
    "lat_long_bg_no_zip_before_2000_2": 0,
    "lat_long_bg_zip_after_2000": 0,
    "lat_long_no_bg_zip_after_2000":0,
    "final_lat_long": 0,
    "lat_long_zip_after_2000" : 0,
    "lat_long_only_1" :0
}

### Creating new geometry

In [68]:
# Create an empty GeoDataFrame to store the intersection results
new_unit_df = gpd.GeoDataFrame(columns=['reportedZipCode', 'countyCode', 'censusTract','censusBlockGroupFips', 'latitude', 
                                        'longitude', 'state', 'new_geographic_unit', 'year', 'geometry_zipcode',
                                       'geometry_county', 'geometry_tract','geometry_BG','geometry_lat_long','geometry_state', 
                                        'geometry_new_unit', 'cbgInconsistent', 'tractInconsistent', 'countyInconsistent', 
                                        'stateInconsistent', 'latlongInconsistent', 'zipcodeInconsistent','multipleinconsistent','singleinconsistent'])

empty = Polygon()

# Iterate through each row in BG_df_1990 and each row in lat_long_df to find intersections
for idx_unit, row_unit in df_geographic_unique.iterrows():
    year = row_unit['yearOfLoss_1990_2021']
    bg_id = row_unit['censusBlockGroupFips']
    bg_geometry = row_unit['geometry_BG']
    tract_id = row_unit['censusTract']
    tract_geometry = row_unit['geometry_tract']
    county_id = row_unit['countyCode']
    county_geometry = row_unit['geometry_county']
    state = row_unit['state']
    state_geometry = row_unit['geometry_state']
    lat_long_geometry = row_unit['geometry_lat_long']
    lat = row_unit['latitude']
    long = row_unit['longitude']
    zipcode_geometry = row_unit['geometry_zipcode']
    zipcode = row_unit['reportedZipCode']
    cbgInconsistent = row_unit['cbgInconsistent']
    tractInconsistent = row_unit['tractInconsistent']
    countyInconsistent = row_unit['countyInconsistent']
    stateInconsistent = row_unit['stateInconsistent']
    latlongInconsistent = row_unit['latlongInconsistent']
    zipcodeInconsistent = row_unit['zipInconsistent']
    multipleInconsistent = row_unit['multiple_inconsistent']
    singleInconsistent = row_unit['single_inconsistent']
    new_geographic_unit = '0'
    intersection_geometry = empty

    if (singleInconsistent == 1) and (latlongInconsistent == 1):

        if (bg_geometry is not None) and (zipcode_geometry is not None) and (year >= 2000):
            intersection_geometry = bg_geometry.intersection(zipcode_geometry)
            assignment_counts["bg_zip_not_empty"] += 1


            if intersection_geometry.is_empty:
                print("OH NO!!")
                break
            else:
                print("bg zip not empty")
                new_geographic_unit = f"{'0'*4}{'0'*4}{bg_id:12}{zipcode:5}{year:4}"
                
        elif (bg_geometry is not None) and ((zipcode_geometry is None) or (year < 2000)):
            new_geographic_unit = f"{'0'*4}{'0'*4}{bg_id:12}{'0'*5}{'0'*4}"
            assignment_counts["bg_no_zip_before_2000"] += 1
            intersection_geometry = bg_geometry
                
        elif (bg_geometry is None) and (zipcode_geometry is not None) and (year >= 2000):
            new_geographic_unit = f"{'0'*4}{'0'*4}{'0'*12}{zipcode:5}{year:4}"
            assignment_counts["no_bg_zip_after_2000"] += 1
            intersection_geometry = zipcode_geometry

        
        else:
            new_geographic_unit = f"{'0'*4}{'0'*4}{'0'*12}{'0'*5}{'0'*4}"
            assignment_counts["no_bg_no_zip"] += 1
            intersection_geometry = empty
                
            
            
    else:     
  
        # Check for the lat_long_geometry being None
        if lat_long_geometry is None:
            new_geographic_unit = f"{'0'*4}{'0'*4}{'0'*12}{'0'*5}{'0'*4}"
            assignment_counts["lat_long_none"] += 1
            intersection_geometry = empty
    
        elif (bg_geometry is None and (zipcode_geometry is None or (zipcode_geometry is not None and year < 2000))):
            new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{'0'*12}{'0'*5}{'0'*4}"
            assignment_counts["lat_long_only_1"] += 1
            intersection_geometry = lat_long_geometry

        elif ((zipcode_geometry is None or (zipcode_geometry is not None and year < 2000)) and not is_empty_or_none(lat_long_geometry.intersection(bg_geometry))):
            new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{bg_id:12}{'0'*5}{'2010':4}"
            assignment_counts["lat_long_bg_no_zip_before_2000_1"] += 1
            intersection_geometry = lat_long_geometry.intersection(bg_geometry)

        elif (bg_geometry is None and zipcode_geometry is not None and year >= 2000 and not is_empty_or_none(lat_long_geometry.intersection(zipcode_geometry))):
            new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{'0'*12}{zipcode:5}{year:4}"
            assignment_counts["lat_long_zip_after_2000"] += 1
            intersection_geometry = lat_long_geometry.intersection(zipcode_geometry)

    
        elif (bg_geometry is not None and zipcode_geometry is not None and year >= 2000):
            if not is_empty_or_none(lat_long_geometry.intersection(bg_geometry).intersection(zipcode_geometry)):
                new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{bg_id:12}{zipcode:5}{year:4}"
                assignment_counts["lat_long_bg_zip_after_2000"] += 1
                intersection_geometry = lat_long_geometry.intersection(bg_geometry).intersection(zipcode_geometry)
        
            elif not is_empty_or_none(lat_long_geometry.intersection(bg_geometry)):
                new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{'0'*12}{'0'*5}{'0'*4}"
                assignment_counts["lat_long_bg_no_zip_before_2000_2"] +=1
                intersection_geometry = lat_long_geometry.intersection(bg_geometry)
        
            elif not is_empty_or_none(lat_long_geometry.intersection(zipcode_geometry)):
                new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{'0'*12}{zipcode:5}{year:4}"
                assignment_counts['lat_long_no_bg_zip_after_2000'] +=1
                intersection_geometry = lat_long_geometry.intersection(zipcode_geometry)
            else:
                new_geographic_unit = f"{'0' if lat >= 0 else '1'}{int(abs(lat) * 10):3}{'0' if long >= 0 else '1'}{int(abs(long) * 10):3}{'0'*12}{'0'*5}{'0'*4}"
                assignment_counts['final_lat_long'] +=1
                intersection_geometry = lat_long_geometry

    
    new_unit_df = pd.concat([new_unit_df, pd.DataFrame({
        'reportedZipCode': [zipcode],
        'countyCode': [county_id],
        'censusTract': [tract_id],
        'censusBlockGroupFips': [bg_id],
        'latitude': [lat],
        'longitude': [long],
        'state': [state],
        'new_geographic_unit': [new_geographic_unit],
        'year': [year],
        'geometry_zipcode': [zipcode_geometry],
        'geometry_county': [county_geometry],
        'geometry_tract': [tract_geometry],
        'geometry_BG': [bg_geometry],
        'geometry_lat_long': [lat_long_geometry],
        'geometry_state': [state_geometry],
        'geometry_new_unit': [intersection_geometry],
        'cbgInconsistent': [cbgInconsistent],
        'tractInconsistent': [tractInconsistent],
        'countyInconsistent': [countyInconsistent],
        'stateInconsistent': [stateInconsistent],
        'latlongInconsistent': [latlongInconsistent],
        'zipcodeInconsistent': [zipcodeInconsistent],
        'multipleInconsistent': [multipleInconsistent],
        'singleInconsistent': [singleInconsistent]
        })], ignore_index=True)

## Visualization

In [69]:
print(assignment_counts)

{'bg_zip_not_empty': 0, 'bg_no_zip_before_2000': 0, 'no_bg_zip_after_2000': 0, 'no_bg_no_zip': 0, 'lat_long_none': 13351, 'lat_long_bg_no_zip_before_2000_1': 108106, 'lat_long_bg_no_zip_before_2000_2': 5583, 'lat_long_bg_zip_after_2000': 164143, 'lat_long_no_bg_zip_after_2000': 77, 'final_lat_long': 8, 'lat_long_zip_after_2000': 6689}


In [70]:
len(df_geographic_unique)

300403

In [92]:
total = sum(assignment_counts.values())
print(total)

297957


In [87]:
sum((new_unit_df['new_geographic_unit'].isnull()) | (new_unit_df['new_geographic_unit'] == ''))


0

In [97]:
total_counts = new_unit_df['new_geographic_unit'].value_counts().sum()
print(total_counts)

300403


In [91]:
len(new_unit_df)

300403

In [93]:
df_geographic_unique['geometry_lat_long'].isnull().sum() + 297957


299951

1994

In [ ]:
visualize_NC = lat_long_df[(lat_long_df['state'] == 'NC') & (lat_long_df['longitude'] < 174.0) & (lat_long_df['longitude'] < 174.0) & (lat_long_df['latitude'] < 40.0)]

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(visualize_NC, geometry='geometry')

# Set the current CRS to WGS 84 (assuming your data is in latitude and longitude)
gdf.crs = "EPSG:4326"

# Project the GeoDataFrame to a suitable CRS (using Web Mercator here for demonstration)
gdf_projected = gdf.to_crs(epsg=3857)

# Compute the average area in square meters
average_area_m2 = gdf_projected.geometry.area.mean()
# Convert the area from square meters to square kilometers
average_area_km2 = average_area_m2 / 1e6
print(f"Average Area: {average_area_km2} square kilometers")

# Plot using the 'viridis' colormap
gdf.plot(edgecolor="black", cmap='Reds', figsize=(10, 10))

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('NC Boundary: Using Latitude-Longitude geometries')

plt.show()

In [ ]:
visualize_NC = BG_df[(BG_df['state'] == 'NC') & (BG_df['year'] == 2010)]
# Find the centroid for each geometry
centroids = visualize_NC.geometry.centroid

# Filter the rows where the y-coordinate (latitude) of the centroid is greater than 42
visualize_NC = visualize_NC[centroids.y < 37]
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(visualize_NC, geometry='geometry')

# Set the current CRS to WGS 84 (assuming your data is in latitude and longitude)
gdf.crs = "EPSG:4326"

# Project the GeoDataFrame to a suitable CRS (using Web Mercator here for demonstration)
gdf_projected = gdf.to_crs(epsg=3857)

# Compute the average area in square meters
average_area_m2 = gdf_projected.geometry.area.mean()
# Convert the area from square meters to square kilometers
average_area_km2 = average_area_m2 / 1e6
print(f"Average Area: {average_area_km2} square kilometers")

# Plot using the 'viridis' colormap
gdf.plot(edgecolor="black", cmap='Reds', figsize=(10, 10))

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('NC Boundary: Using Census BlockGroup geometries for 2010')

plt.show()

In [ ]:
visualize_NC = zipcode_df[(zipcode_df['state'] == 'NC') & (zipcode_df['year'] == 2010)]

centroids = visualize_NC.geometry.centroid

# Filter the rows where the y-coordinate (latitude) of the centroid is greater than 42
visualize_NC = visualize_NC[centroids.y < 37]
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(visualize_NC, geometry='geometry')

# Set the current CRS to WGS 84 (assuming your data is in latitude and longitude)
gdf.crs = "EPSG:4326"

# Project the GeoDataFrame to a suitable CRS (using Web Mercator here for demonstration)
gdf_projected = gdf.to_crs(epsg=3857)

# Compute the average area in square meters
average_area_m2 = gdf_projected.geometry.area.mean()
# Convert the area from square meters to square kilometers
average_area_km2 = average_area_m2 / 1e6
print(f"Average Area: {average_area_km2} square kilometers")

# Plot using the 'viridis' colormap
gdf.plot(edgecolor="black", cmap='Reds', figsize=(10, 10))

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('NC Boundary: Using Zipcode geometries for 2010')

plt.show()

In [ ]:
visualize_NC = new_unit_df[(zipcode_df['state'] == 'NC') & (new_unit_df['year'] == 2010)]

centroids = visualize_NC.geometry.centroid

# Filter the rows where the y-coordinate (latitude) of the centroid is greater than 42
visualize_NC = visualize_NC[centroids.y < 37]
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(visualize_NC, geometry='geometry')

# Set the current CRS to WGS 84 (assuming your data is in latitude and longitude)
gdf.crs = "EPSG:4326"

# Project the GeoDataFrame to a suitable CRS (using Web Mercator here for demonstration)
gdf_projected = gdf.to_crs(epsg=3857)

# Compute the average area in square meters
average_area_m2 = gdf_projected.geometry.area.mean()
# Convert the area from square meters to square kilometers
average_area_km2 = average_area_m2 / 1e6
print(f"Average Area: {average_area_km2} square kilometers")

# Plot using the 'viridis' colormap
gdf.plot(edgecolor="black", cmap='Reds', figsize=(10, 10))

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('NC Boundary: Using new_geographic_unit geometries for 2010')

plt.show()